In [1]:
//CONTEXTE DE TRAVAIL
import org.apache.spark.sql.SparkSession
import org.apache.spark._
import org.apache.spark.rdd._
import org.apache.spark.SparkContext._
import org.apache.spark.mllib.feature.HashingTF
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.classification.{SVMModel, SVMWithSGD}
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import scala.util.{Success, Try}
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.sql.functions



//LECTURE DU FICHIER D'ENTREE DES COMMENTAIRES AVEC DELIMITEUR #
val commentaires_bruts = spark.read.option("delimiter", "\t")
                                    .option("multiline",true)
                                    .csv("Data/test_data.csv")

val stopwords = spark.sparkContext.textFile("French_stop_words.txt").collect()
val remover = new StopWordsRemover()
      .setStopWords(stopwords)
      .setInputCol("_c1")
      .setOutputCol("removed")

val toArray = udf[Array[String], String]( _.split("[ '-.,;:?!\\(\\)]"))
val commentaires_toarray = commentaires_bruts.withColumn("_c1", toArray(lower(commentaires_bruts("_c1"))))
val commentaires_withoutSW = remover.transform(commentaires_toarray)  

//CREATION DE LA TABLE ASSOCIEE AUX COMMENTAIRES LUS
commentaires_withoutSW.createOrReplaceTempView("commentaires_withoutSW")

//CREATION DU NOM DES COLONNES POUR LA TABLE COMMENTAIRES
val commentaires = spark.sql("select _c0 as Id_comment, concat_ws(' ',_c1) as commentaire,concat_ws(' ',removed) as commentaire_SW, _c2 as qualite from commentaires_withoutSW")

//CREATION DE LA TABLE ASSOCIEE AUX COMMENTAIRES FINALISES
commentaires.createOrReplaceTempView("commentaires")


commentaires_bruts = [_c0: string, _c1: string ... 3 more fields]
stopwords = Array(alors, au, aucun, aussi, autre, avant, avec, avoir, car, ce, cela, ces, ceux, ...


[alors, au, aucun, aussi, autre, avant, avec, avoir, car, ce, cela, ces, ceux, chaque, ci, comme, comment, dans, des, du, dedans, dehors, depuis, donc, elle, elles, en, encore, est, et, eu, hors, ici, il, ils, je, la, le, les, leur, là, ma, maintenant, mais, mes, mien, moins, mon, même, ni, notre, nous, or, ou, où, par, parce, parole, peut, plupart, pour, pourquoi, quand, que, quel, quelle, quelles, quels, qui, sa, ses, si, sien, sur, ta, tandis, tels, tes, tien, ton, tous, tout, tu, votre, vous, vu, ça, été, être, à, ont, pu, a, !, ?, ., ;, -, une, l'ai, j'ai, aux, (, ), d'en, y, ce, sera, d'une, de, suis, était, un, e, qu, j, l, m, a, on, fait, ]

In [2]:
commentaires.take(10)

1,hote extremement attentif et tres attache à remettre en ordre les petits soucis mineurs qui ont pu apparaitre studio sympa et tres bien placea recommander,hote extremement attentif tres attache remettre ordre petits soucis mineurs apparaitre studio sympa tres bien placea recommander,positif
2,aucun probleme durant le sejour vincent fût toujours à l heure ainsi que disponible pour repondre à mes questions le logement etait parfaitement fonctionnel et tres cozy situe dans le 6e ce qui ajoute bien sûr un cadre agreable a recommander,probleme durant sejour vincent fût toujours heure ainsi disponible repondre questions logement etait parfaitement fonctionnel tres cozy situe 6e ajoute bien sûr cadre agreable recommander,positif
3,un accueil chaleureux de kathy dans cet appartement agreable et tres bien place dans paris et une organisation tres professionnelle et attentive de la part de cobblestone une adresse à recommander,accueil chaleureux kathy cet appartement agreable tres bien place paris organisation tres professionnelle attentive part cobblestone adresse recommander,positif
4,j ai ete accueilli en debut d apres midi au studio par la sympathique katia qui m a fourni tous les renseignements necessaires à mon court sejour je l ai par ailleurs revue une fois où elle est venue m apporter les billets pour la croisiere sur la seine que je ne parvenais pas à imprimer le studio est totalement conforme aux photos et à la description qui en est faite situe au 3em etage avec ascenseur d une residence securisee donnant sur une petite place il est dans un quartier tres vivant et agreable à deux pas du marais où tout semble à portee de main de la boulangerie mitoyenne aux bars et restaurants voisins en passant par la pharmacie d en face 3 stations de metro tres proches reglent facilement la question du deplacement petit mais fonctionnel tres lumineux et bien decore on y revient volontiers apres les excursions de la journee le seul petit bemol sera pour les voisins de l immeuble d en face qui auraient dû en mettre plusieurs à la clef lors d une fete assez bruyante malgre ce qui peut arriver partout ailleurs dans une grande ville je le recommande vivement et serais pret à y revenir volontiers,ai ete accueilli debut d apres midi studio sympathique katia fourni renseignements necessaires court sejour ai ailleurs revue fois venue apporter billets croisiere seine ne parvenais pas imprimer studio totalement conforme photos description faite situe 3em etage ascenseur d residence securisee donnant petite place quartier tres vivant agreable deux pas marais semble portee main boulangerie mitoyenne bars restaurants voisins passant pharmacie d face 3 stations metro tres proches reglent facilement question deplacement petit fonctionnel tres lumineux bien decore revient volontiers apres excursions journee seul petit bemol voisins immeuble d face auraient dû mettre plusieurs clef lors d fete assez bruyante malgre arriver partout ailleurs grande ville recommande vivement serais pret revenir volontiers,positif
5,merci à l’hote pour sa ponctualite à nous accueillir et les tres bonnes indications pour s y rendre la localisation est vraiment excellente tres centrale endroit tres tranquille attention il faut bien considerer que la photo du studio est prise avec un grand angle les dimensions reelles sont surprenantes l appart est plutot sombre les fenetres donnant sur cour interieure exiguë rapport qualite/prix interessant,merci l’hote ponctualite accueillir tres bonnes indications s rendre localisation vraiment excellente tres centrale endroit tres tranquille attention faut bien considerer photo studio prise grand angle dimensions reelles sont surprenantes appart plutot sombre fenetres donnant cour interieure exiguë rapport qualite/prix interessant,positif
6,nous avons ete tres bien accueilli à notre arrivee excellent timing remise des clefs et explications des points essentiels de la procedure studio tres calme et sûr en plein coeur de paris on sent que le pr

In [3]:
val commentaires_negatifs = spark.sql("select * from commentaires where qualite like '%negatif%'") 
val commentaires_positifs = spark.sql("select * from commentaires where qualite like '%positif%'")
val commentaires_positifs_nbre = commentaires_positifs.count()
val commentaires_negatifs_nbre = commentaires_negatifs.count()


//DEFINITION DE LA TAILLE DE L'ECHANTILLON D'APPRENTISSAGE
val taille_echantillon = Math.min(commentaires_negatifs_nbre, commentaires_positifs_nbre).toInt
//DEFINITION DE L'ECHANTILLON D'APPRENTISSAGE AVEC AUTANT DE COMMENTAIRES POSITIFS QUE NEGATIFS
var echantillon = commentaires_positifs.limit(taille_echantillon).unionAll(commentaires_negatifs.limit(taille_echantillon))
echantillon.createOrReplaceTempView("echantillon")
val echantillon_msg = spark.sql("select concat( qualite, ' ', commentaire_SW) as msg from echantillon")
val echantillonRDD = echantillon_msg.rdd
val hashingTF = new HashingTF(20000)


commentaires_negatifs = [Id_comment: string, commentaire: string ... 2 more fields]
commentaires_positifs = [Id_comment: string, commentaire: string ... 2 more fields]
commentaires_positifs_nbre = 187708
commentaires_negatifs_nbre = 1330
taille_echantillon = 1330
echantillon = [Id_comment: string, commentaire: string ... 2 more fields]
echantillon_msg = [msg: string]
echantillonRDD = MapPartitionsRDD[33] at rdd at <console>:60
hashingTF = org.apache.spark.mllib.feature.HashingTF@b39329c


org.apache.spark.mllib.feature.HashingTF@b39329c

In [4]:
println("Commentaires                  = "+commentaires.count())
println("   dont commentaires négatifs = "+commentaires_negatifs.count())
println("   dont commentaires positifs = "+commentaires_positifs.count())
println("Taille echantillon = "+echantillonRDD.count())


Commentaires                  = 205581
   dont commentaires négatifs = 1330
   dont commentaires positifs = 187708
Taille echantillon = 2660


In [5]:
val echantillon_reformat = echantillonRDD.map(
  row =>{
    Try{
      val msg = row.toString.toLowerCase()
      var isHappy:Int = 0
      if(msg.contains("positif ")){
        isHappy = 1
      }else if(msg.contains("negatif ")){
        isHappy = 0
      }
      var msgSanitized = msg.replaceAll("positif ", "")
      msgSanitized = msgSanitized.replaceAll("negatif ","")
      //Return a tuple
      (isHappy, msgSanitized.split(" ").toSeq)
    }
  }
)

var echantillon_ok = echantillon_reformat.filter((_.isSuccess)).map(_.get)
val echantillon_transfo = echantillon_ok.map(
  t => (t._1, hashingTF.transform(t._2)))
  .map(x => new LabeledPoint((x._1).toDouble, x._2))

var sample = echantillon_ok.take(1000).map(
  t => (t._1, hashingTF.transform(t._2), t._2))
  .map(x => (new LabeledPoint((x._1).toDouble, x._2), x._3))

val splits = echantillon_transfo.randomSplit(Array(0.7, 0.3))
val (trainingData, validationData) = (splits(0), splits(1))



echantillon_reformat = MapPartitionsRDD[52] at map at <console>:52
echantillon_ok = MapPartitionsRDD[54] at map at <console>:70
echantillon_transfo = MapPartitionsRDD[56] at map at <console>:73
sample = Array(((1.0,(20000,[1113,1308,2499,3680,4652,6629,8129,8431,8809,9927,10070,12919,13314,16011,17211,18171],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0])),WrappedArray([hote, extremement, attentif, tres, attache, remettre, ordre, petits, soucis, mineurs, apparaitre, studio, sympa, tres, bien, placea, recommander])), ((1....


[((1.0,(20000,[1113,1308,2499,3680,4652,6629,8129,8431,8809,9927,10070,12919,13314,16011,17211,18171],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0])),WrappedArray([hote, extremement, attentif, tres, attache, remettre, ordre, petits, soucis, mineurs, apparaitre, studio, sympa, tres, bien, placea, recommander])), ((1.0,(20000,[727,2499,2718,3982,5501,8809,9007,9950,10173,10430,10773,11139,11735,11779,12585,12627,12951,13443,13842,16046,17213,18171,18747,18881,19637],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])),WrappedArray([probleme, durant, sejour, vincent, fût, toujours, heure, ainsi, disponible, repondre, questions, logement, etait, parfaitement, fonctionnel, tres, cozy, situe, 6e, ajoute, bien, sûr, cadre, agreable, recommander])), ((1.0,(20000,[2371,2499,2966,3930,4295,6270,7200,8740,8809,10773,10981,18171,18191,18944,19479,19627,19948],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0]

In [6]:
//CONSTRUCTION DU MODELE A PARTIR DU JEU D'APPRENTISSAGE

val numIterations = 100
val model = SVMWithSGD.train(trainingData, numIterations)

//EVALUATION DU MODELE
// Clear the default threshold.
model.clearThreshold()
// Save and load model
model.save(sc, "modele/SVMClassification")


// Application du modèle au jeu de validation
val scoreAndLabels = validationData.map { point =>
  val score = model.predict(point.features)
  (score, point.label)
}



[Stage 330:>                                                        (0 + 2) / 2]Area under ROC = 0.9675730519480499


numIterations = 100
model = org.apache.spark.mllib.classification.SVMModel: intercept = 0.0, numFeatures = 20000, numClasses = 2, threshold = None
scoreAndLabels = MapPartitionsRDD[262] at map at <console>:60
metrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@642763d1
auROC = 0.9675730519480499


0.9675730519480499

In [5]:
val predictionAndLabel = validationData.map(p => (model.predict(p.features), p.label))


Name: org.apache.spark.SparkException
Message: Task not serializable
StackTrace:   at org.apache.spark.util.ClosureCleaner$.ensureSerializable(ClosureCleaner.scala:298)
  at org.apache.spark.util.ClosureCleaner$.org$apache$spark$util$ClosureCleaner$$clean(ClosureCleaner.scala:288)
  at org.apache.spark.util.ClosureCleaner$.clean(ClosureCleaner.scala:108)
  at org.apache.spark.SparkContext.clean(SparkContext.scala:2287)
  at org.apache.spark.rdd.RDD$$anonfun$map$1.apply(RDD.scala:370)
  at org.apache.spark.rdd.RDD$$anonfun$map$1.apply(RDD.scala:369)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
  at org.apache.spark.rdd.RDD.withScope(RDD.scala:362)
  at org.apache.spark.rdd.RDD.map(RDD.scala:369)
  ... 50 elided
Caused by: java.io.NotSerializableException: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
Serialization stack:
	- object not serializable (c

In [3]:
import org.apache.spark.mllib.evaluation.MulticlassMetrics
 
val metrics = new MulticlassMetrics(scoreAndLabels)
val cfm = metrics.confusionMatrix
 
val tn = cfm(0, 0)
val fp = cfm(0, 1)
val fn = cfm(1, 0)
val tp = cfm(1, 1)


// Confusion matrix
val confusion = metrics.confusionMatrix
val fprINSAT = 100 * (metrics.falsePositiveRate(0) - metrics.falsePositiveRate(0) % 0.0001) 
val fprSAT = 100 * (metrics.falsePositiveRate(1) - metrics.falsePositiveRate(1) % 0.0001) 

import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

val metrics2 = new BinaryClassificationMetrics(scoreAndLabels)

// AUPRC
val auPRC = 100 * (metrics2.areaUnderPR() - metrics2.areaUnderPR() % 0.0001) 

// AUROC
val auROC = 100 * (metrics2.areaUnderROC() - metrics2.areaUnderROC() % 0.0001) 

[Stage 388:======================================>                  (2 + 1) / 3]

metrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@a0c81fa
cfm = 
tn = 0.0
fp = 0.0
fn = 0.0
tp = 0.0
confusion = 
fprINSAT = 0.0
fprSAT = 0.0
metrics2 = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@145d345
auPRC = 97.31
auROC = 96.75


96.75

In [4]:
println("Modèle SVM")
printf(s"""|=================== Confusion matrix ==========================
           |_____________| %-15s                     %-15s
           |-------------+-------------------------------------------------
           |Predicted = 0| %-15f                     %-15f
           |Predicted = 1| %-15f                     %-15f
           |===============================================================
         """.stripMargin, "Actual = 0", "Actual = 1", tn, fp, fn, tp)
println()
println(s"Taux de Faux positifs pour les insatisfaits = $fprINSAT %")
println(s"Taux de Faux posisifs pour les satisfaits = $fprSAT %")

println(s"Area under precision-recall curve = $auPRC %")
println(s"Area under ROC = $auROC %")

Modèle SVM
=================== Confusion matrix ==========================
_____________| Actual = 0                          Actual = 1     
-------------+-------------------------------------------------
Predicted = 0| 0,000000                            0,000000       
Predicted = 1| 0,000000                            0,000000       
         
Taux de Faux positifs pour les insatisfaits = 0.0 %
Taux de Faux posisifs pour les satisfaits = 0.0 %
Area under precision-recall curve = 97.31 %
Area under ROC = 96.75 %
